# Skin Cancer Classification Challenge

**Summary : ** skin cancer.

## Preliminaries and Imports

In [1]:
#general & system\n",
import os

#data processing
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['figure.figsize'] = 10, 10  #default setting
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

#data augmentation
from PIL import Image
from random import choice
import cv2
import numpy as np
#import keras.preprocessing.image as prep

#ML part
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input
from keras.preprocessing import image as kimage
from keras.models import Sequential
from keras.layers import GlobalMaxPooling2D, Dense, BatchNormalization, GlobalAveragePooling2D, Dropout,Activation,Flatten
from keras.models import Model
from keras.optimizers import SGD
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator

#validation
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight

%matplotlib inline

Using TensorFlow backend.


## I. Load Data

- **Rq : ** Datasets are quite light, can be fully loaded in a laptop memory with ease.

In [2]:
%%time
meta = pd.read_csv("data/metadata.csv")

CPU times: user 36 ms, sys: 0 ns, total: 36 ms
Wall time: 34.6 ms


In [3]:
%%time
filelist = os.listdir("data/resized/")
X_train = np.array([np.array(Image.open("data/resized/"+fname)) for fname in filelist])

CPU times: user 25.5 s, sys: 1.9 s, total: 27.4 s
Wall time: 27.4 s


In [4]:
X_train.shape

(13786, 299, 299, 3)

In [5]:
#get target
filelist = [x.replace(".jpg","") for x in filelist]
strates = [meta[meta["name"]==x]["meta_clinical_benign_malignant"].values[0] for x in filelist]
Y_train = [1 if x=="malignant" else 0 for x in strates]

In [6]:
#control
print(filelist[0:5])
print(Y_train[0:5])
print(strates[0:5])

['ISIC_0010006', 'ISIC_0006014', 'ISIC_0007816', 'ISIC_0000256', 'ISIC_0000680']
[0, 0, 0, 0, 0]
['benign', 'benign', 'benign', 'benign', 'benign']


## II. Data Preparation

- **Resampling**

- **Class weights**

In [7]:
class_weight = class_weight.compute_class_weight('balanced', np.unique(Y_train), Y_train)

- **Data splitting**

In [8]:
#data splitting
X_train, X_valid, Y_train, Y_valid = train_test_split(X_train, Y_train, random_state=42, train_size=0.80,
                                                      stratify = strates)

## III. Deep Learning with Transfer Learning

In [9]:
weights_path = 'model/checkpoints/'

if not os.path.exists(weights_path):
    os.makedirs(weights_path)
    
epochs = 500
batch_size = 5

- **Using VGG16 : **

In [10]:
#load base model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(299,299,3))
base_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 150, 150, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 150, 150, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 150, 150, 64) 0           bn_conv1[0][0]                   
__________________________________________________________________________________________________
max_poolin

In [11]:
#adding top layers
    #sequentialy
add_model = Sequential()
add_model.add(GlobalAveragePooling2D(input_shape=base_model.output_shape[1:])) #Flatten/GlobalAveragePooling2D
add_model.add(Dense(1024, activation='relu'))
add_model.add(Dropout(0.25))
add_model.add(Dense(512, activation='relu'))
add_model.add(Dropout(0.25))
add_model.add(Dense(1, activation='sigmoid'))

model = Model(inputs=base_model.input, outputs=add_model(base_model.output))
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 150, 150, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 150, 150, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 150, 150, 64) 0           bn_conv1[0][0]                   
__________________________________________________________________________________________________
max_poolin

In [ ]:
#freeze lower layers of the model
#for layer in model.layers[:]:
    #layer.trainable = False
for layer in model.layers[0:]:
    layer.trainable = True
    
#compile

from keras import backend as K

def sensitivity(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return true_positives / (possible_positives + K.epsilon())

def specificity(y_true, y_pred):
    true_negatives = K.sum(K.round(K.clip((1-y_true) * (1-y_pred), 0, 1)))
    possible_negatives = K.sum(K.round(K.clip(1-y_true, 0, 1)))
    return true_negatives / (possible_negatives + K.epsilon())

#model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='binary_crossentropy',
              metrics=[sensitivity, specificity])

In [ ]:
%%time
#Lets define the image transormations that we wan

gen = ImageDataGenerator(horizontal_flip=True,
                         vertical_flip=True,
                         width_shift_range=0.2,
                         height_shift_range=0.2,
                         zoom_range=0.2,
                         rotation_range=40)

val_datagen = ImageDataGenerator()
# Here is the function that merges our two generators
# We use the exact same generator with the same random seed for both the y and angle arrays
def gen_flow_for_one_input(X1, y):
    genX1 = gen.flow(X1, y, batch_size=batch_size, seed=42)
    while True:
        X1i = genX1.next()
        yield X1i[0], X1i[1]

def val_datagen_(X1, y):
    genX1 = val_datagen.flow(X1, y, batch_size=batch_size, seed=42)
    while True:
        X1i = genX1.next()
        yield X1i[0], X1i[1]
#Finally create out generator
gen_flow_train = gen_flow_for_one_input(X_train, Y_train)
#gen_flow_val = gen_flow_for_one_input(X_valid, Y_valid)
gen_flow_val = val_datagen_(X_valid, Y_valid)

from keras.callbacks import EarlyStopping, ModelCheckpoint
epochs_to_wait_for_improve = 50
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=epochs_to_wait_for_improve)
checkpoint_callback = ModelCheckpoint(weights_path + 'BestKerasModelVGG16_flat.h5', monitor='val_loss',
                                      verbose=1, save_best_only=True, mode='min')
#fit the model
model.fit_generator(gen_flow_train, validation_data=(X_valid, Y_valid) #gen_flow_val,  #(X_valid, Y_valid),
                    steps_per_epoch=int(np.ceil(len(X_train)/batch_size)),
                    #validation_steps=int(np.ceil(len(X_valid)/batch_size)),
                    epochs=500, verbose=1, callbacks=[early_stopping_callback, checkpoint_callback],
                    class_weight = class_weight)
                    #class_weights = class_weight

Epoch 1/500
2205/2205 [==============================] - 1423s 645ms/step - loss: 0.2024 - acc: 0.9201 - val_loss: 0.1700 - val_acc: 0.9249
Epoch 2/500
  31/2205 [..............................] - ETA: 19:34 - loss: 0.2224 - acc: 0.8989

## IV. Evaluate Model

In [ ]:
model = load_model(weights_path+"BestKerasModelVGG16_flat.h5")
score = model.evaluate(X_valid, Y_valid, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
#compute log loss
pred = model.predict(X_valid)
log_loss(Y_valid,pred)